# 1次元のナビエストークス方程式

$$\frac{\partial u}{\partial t}+u\frac{\partial u}{\partial x}=-\frac{1}{\rho}\frac{\partial p}{\partial x}+\nu\frac{\partial^2 u}{\partial x^2}+F$$

が、ニュートン流体における流体力学のの基礎式ですが、このような偏微分方程式を扱うのは面倒です。理解をするためのであれば、もう少しわかりやすい簡単な偏微分方程式を対象にするのも良いでしょう。

右辺を全部0にします。\
さらに左辺第二項である移流項である速度を「$c$定数」とします。
 $$\frac{\partial u}{\partial t}+c\frac{\partial u}{\partial x}=0\tag{1}$$

(1)式の離散化は以下のようになります。


$$\frac{u^{n+1}_{i}-u^{n}_{i}}{\Delta t}+c\frac{u^{n}_{i}-u^{n}_{i-1}}{\Delta x}=0$$

これを$u^{n+1}$について解くと、
$$u^{n+1}_{i}=u^{n}_{i}+\frac{c\Delta t}{\Delta x}\big(u^{n}_{i}-u^{n}_{i-1}\big)\tag{2}$$

ここで、$\alpha=\frac{c\Delta t}{\Delta x}$として時間刻み空間の分割幅$\Delta x$と$c$で決まるものとしておきます。

(2)をPythonコードで書いてみましょう。

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation, rc
from IPython.display import HTML

nx = 101
xmax = 2.0
dx = xmax / (nx-1)
nt = 40
c = 1
alpha = 1.5
dt = alpha * (dx/c) 
print('dx=',dx)
print('dt=',dt)


x = np.linspace(0,xmax,nx)

un = []
u = np.ones(nx)
u[int(.5 / dx):int(1 / dx + 1)] = 2

fig = plt.figure(figsize=(8,4))
ims=[]
for n in range(nt): 
    un = u.copy()
    if (nt%1==0):
      im = plt.plot(x,u, "r")
      ims.append(im)
    for i in range(1, nx):
        u[i] = un[i] - c * dt / dx * (un[i] - un[i-1])

plt.grid()
plt.ylim([-0.1,3])
plt.xlabel("x")
plt.ylabel("u(m/s)")   
anim = animation.ArtistAnimation(fig, ims)
rc('animation', html='jshtml')
plt.close()
anim

dx= 0.02
dt= 0.03


# エクセルからパラメータを読み込む 

【作成の方針】

- **プリ処理**\
Pythonコードを用意（ナビエストークス方程式を解く）\
エクセルにパラメータを設定\
エクセルにボタンを設置

- **ソルバー**\
エクセルに設置したボタンを押すとPythonコードが実行

- **ポスト**\
実行した結果、流速ベクトルの画像が作成される

# パラメータにしたい数値を考える

今回の計算結果の肝になる部分は、「alpha」という変数を1より大きくした場合に、計算結果が振動してしまうという点ですので、alphaに関係した部分をパラメータにしておきます。\
※パラメータにするのは「★」をつけた変数です。

```
nx = 101 ★
xmax = 2.0
dx = xmax / (nx-1)
nt = 40
c = 1 ★
alpha = 1.5 ★
dt = alpha * (dx/c) 
nx = 101
```

# Excelの読み方を確認する

In [5]:
import pandas as pd

df_import_file = pd.read_excel('param.xlsm')
df_import_file

,Unnamed: 0,Unnamed: 1
0,パラメータ設定,数値
1,nx,101
2,xmax,2
3,c,1
4,alpha,0.5
5,NaN,NaN
6,dx = xmax/(nx-1),0.02


In [6]:
# indexを変更
df_import_file = pd.read_excel('param.xlsm', index_col=0, header=1)
df_import_file

,数値
パラメータ設定,
nx,101.00
xmax,2.00
c,1.00
alpha,0.50
NaN,NaN
dx = xmax/(nx-1),0.02


In [8]:
df_import_file.index[4:]
#df.drop(import_file.index[4:])

Index([nan, 'dx = xmax/(nx-1)'], dtype='object', name='パラメータ設定')

In [9]:
df_import_file.drop(df_import_file.index[4:])

,数値
パラメータ設定,
nx,101.0
xmax,2.0
c,1.0
alpha,0.5


In [10]:
df_import_file = df_import_file.drop(df_import_file.index[4:])

In [11]:
#確認
print(df_import_file.index[1])
print(df_import_file['数値'])

xmax
パラメータ設定
nx       101.0
xmax       2.0
c          1.0
alpha      0.5
Name: 数値, dtype: float64


In [12]:
df_import_file['数値']

パラメータ設定
nx       101.0
xmax       2.0
c          1.0
alpha      0.5
Name: 数値, dtype: float64

In [13]:
for value in df_import_file['数値']:
    print(value)

101.0
2.0
1.0
0.5


In [14]:
param = []
for value in df_import_file['数値']:
    param.append(value)
    
param

[101.0, 2.0, 1.0, 0.5]

この方法でも良いかもしれませんが、どの値がどの変数に対応しているのかがわかりにくいですよね。\
（作成していてすごく思いました）

今回の場合は、リストでパラメータを設定するのではなく、辞書型でデータを作成する方が「変数名と値」が対応付けられてわかりやすそうです。

In [15]:
# indexを変更
df_import_file = pd.read_excel('param.xlsm', index_col=0, header=1)
df_import_file = df_import_file.drop(df_import_file.index[4:])
df_import_file

,数値
パラメータ設定,
nx,101.0
xmax,2.0
c,1.0
alpha,0.5


In [16]:
param_ = df_import_file.to_dict()
param_

{'数値': {'nx': 101.0, 'xmax': 2.0, 'c': 1.0, 'alpha': 0.5}}

In [17]:
param = param_['数値']
param

{'nx': 101.0, 'xmax': 2.0, 'c': 1.0, 'alpha': 0.5}

In [19]:
print(param['nx'] ,type(param['nx']))
print(param['xmax'],type(param['xmax']))
print(param['c'],type(param['c']))
print(param['alpha'],type(param['alpha']))

101.0 <class 'float'>
2.0 <class 'float'>
1.0 <class 'float'>
0.5 <class 'float'>


# 計算部分にパラメータを代入する

In [20]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation, rc
from IPython.display import HTML

nx = int(param['nx']) #★
xmax = param['xmax'] #★
dx = xmax / (nx-1)
nt = 40
c = param['c'] #★
alpha = param['alpha'] #★
dt = alpha * (dx/c) 


x = np.linspace(0,xmax,nx)

un = []
u = np.ones(nx)
u[int(.5 / dx):int(1 / dx + 1)] = 2

fig = plt.figure(figsize=(8,4))
ims=[]
for n in range(nt): 
    un = u.copy()
    if (nt%1==0):
      im = plt.plot(x,u, "r")
      ims.append(im)
    for i in range(1, nx):
        u[i] = un[i] - c * dt / dx * (un[i] - un[i-1])

plt.grid()
plt.ylim([-0.1,3])
plt.xlabel("x")
plt.ylabel("u(m/s)")   
anim = animation.ArtistAnimation(fig, ims)
rc('animation', html='jshtml')
plt.close()
anim